In [4]:
from glob import glob

import numpy as np
import pandas as pd
import joblib
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import StackingClassifier


from utilities import *

In [5]:
random_state = 42
code_folder_path = ""
data_percentage = 0.001 # how much of the data we will use

In [6]:
class DenseTransformer(TransformerMixin):
    def fit(self, X, y=None, **fit_params):
        return self
    def transform(self, X, y=None, **fit_params):
        return X.todense()

stack_estimators = [
    ('NaiveBayes', make_pipeline(DenseTransformer(), GaussianNB())),
    ('DecisionTree', DecisionTreeClassifier(max_depth=100)),
    ('SVM', SVC(kernel="linear", C=0.6)),
    # ("LogisticRegression", LogisticRegression(max_iter=1000, random_state=random_state))
]

names = [
    "Nearest Neighbors",
    "Linear SVM",
    # "RBF SVM", # Bad performance
#     "Gaussian Process", 
    "Decision Tree",
    "Random Forest",
#     "Neural Net", # Too slow
    "AdaBoost",
    "Naive Bayes",
    # "QDA", # Terrible performance
]

classifiers = [
    KNeighborsClassifier(5),
    StackingClassifier(estimators=stack_estimators, final_estimator=DecisionTreeClassifier(max_depth=5)),
    SVC(kernel="linear", C=0.5),
    # SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=60),
    # RandomForestClassifier(max_depth=5, n_estimators=100),
    # MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    # QuadraticDiscriminantAnalysis(), 
]

NameError: name 'LogisticRegression' is not defined

In [ ]:
df = get_SMADC_folder_data()
df = df.sample(frac=data_percentage)

In [ ]:
%%time
X_train, X_test, y_train, y_test = train_test_split(df["Text"], df["Region"], random_state=random_state)
# count_vectorizer = CountVectorizer()
count_vectorizer = TfidfVectorizer()
X_vectorized = count_vectorizer.fit_transform(X_train)
standard_scaler = StandardScaler(with_mean=False).fit(X_vectorized)

Wall time: 297 ms


In [ ]:
%%time
for name, clf in zip(names, classifiers):
    if name in ("Naive Bayes", "Gaussian Process", "QDA"):
        model = make_pipeline(count_vectorizer, standard_scaler, DenseTransformer(), clf)
    else:
        model = make_pipeline(count_vectorizer, standard_scaler, clf)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(name, score, flush=True)
    joblib.dump(model, join(code_folder_path, f"models/other_models/{name}_acc={score.round(3)}.model"))

Nearest Neighbors 0.4559909142532652
StackClassifier 0.6607041453719478
Linear SVM 0.5857467348097671
Decision Tree 0.5570698466780238
AdaBoost 0.6391254968767746


D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\utils\validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\utils\validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Naive Bayes 0.4514480408858603


D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\utils\validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [ ]:
estimators = [
    ('rf', make_pipeline(count_vectorizer,RandomForestClassifier(max_depth=5, n_estimators=100, random_state=42))),
    ('svr', make_pipeline(count_vectorizer,SVC(gamma=2, C=1))),
    ("ada", make_pipeline(count_vectorizer,AdaBoostClassifier()))
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=DecisionTreeClassifier(max_depth=5)
)
clf.fit(X_train,y_train).score(X_test, y_test)

# Load

In [7]:
loaded_models = [
    "MultinomialNaiveBayes_acc=0.865.model",
    "RandomForest_acc=0.76.model",
    "Linear SVM_acc=0.747.model"
]

In [8]:
dfs = {
    "annotated_data": get_annotated_data_folder_data(),
    "arabic_dialects": get_arabic_dialects_dataset_folder_data(),
    "dart": get_dart_folder_data()
}

In [9]:
results = []
for model_path in loaded_models:
    print(model_path)
    model = joblib.load(join(code_folder_path, "models", "other_models", model_path))
    for name, df in dfs.items():
        df = df.dropna()
        preds = model.predict(df["Text"])
        results.append((
            model_path,
            name,
            model_path[model_path.rindex("=")+1:model_path.rindex(".")], # SMADC Accuracy
            f1_score(df["Region"], preds, average="macro"),
            precision_score(df["Region"], preds, average="macro"),
            recall_score(df["Region"], preds, average="macro")
        ))

MultinomialNaiveBayes_acc=0.865.model


D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 1.0

RandomForest_acc=0.76.model


D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Linear SVM_acc=0.747.model


D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
df_results = pd.DataFrame(results, columns=["Model name", "Dataset", "SMADC accuracy", "Macro F1", "Macro precision", "Macro recall"])
df_results["Model name"] = df_results["Model name"].apply(lambda name: name[:name.find("_")])

In [17]:
print(df_results.to_latex(index=False))

\begin{tabular}{lllrrr}
\toprule
           Model name &         Dataset & SMADC accuracy &  Macro F1 &  Macro precision &  Macro recall \\
\midrule
MultinomialNaiveBayes &  annotated\_data &          0.865 &  0.552954 &         0.561883 &      0.611622 \\
MultinomialNaiveBayes & arabic\_dialects &          0.865 &  0.450884 &         0.464933 &      0.446934 \\
MultinomialNaiveBayes &            dart &          0.865 &  0.737389 &         0.742771 &      0.748739 \\
         RandomForest &  annotated\_data &           0.76 &  0.462519 &         0.497453 &      0.497691 \\
         RandomForest & arabic\_dialects &           0.76 &  0.357108 &         0.400919 &      0.357486 \\
         RandomForest &            dart &           0.76 &  0.667313 &         0.699270 &      0.681225 \\
           Linear SVM &  annotated\_data &          0.747 &  0.491982 &         0.524889 &      0.523377 \\
           Linear SVM & arabic\_dialects &          0.747 &  0.373722 &         0.418487 &      0